In [1]:
#インポート文
import numpy as np
import pandas as pd

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

import lightgbm as lgb
from lightgbm import LGBMRanker

from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.model_selection import KFold

import optuna

In [2]:
#データのインポート
race_df = pd.read_csv("../data/main/race_data.csv",parse_dates=['datetime'])
horse_df = pd.read_csv("../data/main/horse_data.csv")

In [3]:
#データのマージ
race_df_for_merge = race_df[['race_id',
                             "race_round",
                             "race_title",
                             "weather",
                             "ground_status",
                             "where_racecourse",
                             "total_horse_number",
                             "frame_number_first",
                             "horse_number_first",
                             "frame_number_second",
                             "horse_number_second",
                             "frame_number_third",
                             "horse_number_third",
                             "tansyo",
                             "hukusyo_first",
                             "hukusyo_second",
                             "hukusyo_third",
                             "wakuren",
                             "umaren",
                             "wide_1_2",
                             "wide_1_3",
                             "wide_2_3",
                             "umatan",
                             "renhuku3",
                             "rentan3",
                             "is_obstacle",
                             "ground_type",
                             "is_left_right_straight",
                             "distance",
                             "weather_rain",
                             "weather_snow",
                             "datetime"
                            ]]

merged_horse_df = pd.merge(horse_df, race_df_for_merge, on='race_id')

In [4]:
#datetimeをUnix時間に変換してint32に変換する
unix_time = merged_horse_df['datetime'].astype('int64')
unix_time_int32 = unix_time.astype('int32')

merged_horse_df['datetime'] = unix_time_int32

In [5]:
#エンコード
for column in ['race_title',
               'weather',
               'where_racecourse',
               'ground_type',
               'is_left_right_straight']:
    le = LabelEncoder()
    le.fit(merged_horse_df[column])
    merged_horse_df[column] = le.transform(merged_horse_df[column])

In [6]:
#使う特徴量の設定
id = {"race_id",
      "horse_id"
    }

feature = [
#horse_df
           "race_id",#後で抜く
           "horse_id",#後で抜く
#            "rank",#目的変数
    
    
           "frame_number",
           "horse_number",
           "sex_and_age",
           "burden_weight",
           "rider_id",
#            "goal_time",結果なので
#            "goal_time_dif",結果なので
#            "half_way_rank",結果なので
#            "last_time",結果なので
#            "odds",
#            "popular",
           "horse_weight",
           "tamer_id",
           "owner_id",
           "is_down",
           "is_senba",
           "is_mesu",
           "is_osu",
           "horse_weight_dif",
           "burden_weight_rate",
#            "avg_velocity",結果なので

#race_df           
             "race_round",
#              "race_title",タイトルはややこしいので
             "weather",
             "ground_status",
             "where_racecourse",
             "total_horse_number",
#              "frame_number_first",結果なので
#              "horse_number_first",
#              "frame_number_second",
#              "horse_number_second",
#              "frame_number_third",
#              "horse_number_third",
#              "tansyo",
#              "hukusyo_first",
#              "hukusyo_second",
#              "hukusyo_third",
#              "wakuren",
#              "umaren",
#              "wide_1_2",
#              "wide_1_3",
#              "wide_2_3",
#              "umatan",
#              "renhuku3",
#              "rentan3",
             "is_obstacle",
             "ground_type",
             "is_left_right_straight",
             "distance",
             "weather_rain",
             "weather_snow",
             "datetime"
            ]

target = ['rank']

X_id = merged_horse_df[feature]
y = merged_horse_df[target]

In [7]:
#スコア結果を入れるリスト
reports = []
auc_scores = []
precision_scores = []
recall_scores = []
f1_scores = []
support_scores = []

#予測結果を入れるリスト
pred_df_list = []

def objective(trial):
    
# Optuna
    # ハイパーパラメータの探索範囲
    params = {
        'boosting_type': 'gbdt',
        'num_leaves': trial.suggest_int('num_leaves', 10, 100),
        'learning_rate': trial.suggest_float('learning_rate', 1e-4, 1e-1),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.1, 1.0),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.1, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 10),
        'lambda_l1': trial.suggest_float('lambda_l1', 1e-8, 10.0),
        'lambda_l2': trial.suggest_float('lambda_l2', 1e-8, 10.0),
        'verbose': -1
    }
    
    #LGBMRankerの設定
    ranker = LGBMRanker(**params)

# K-fold
    #kflodの設定
    kfold = KFold(n_splits=5, shuffle=True, random_state=42)

    #kflodで学習用と検証用に分ける
    for fold, (train_idx, val_idx) in enumerate(kfold.split(X_id, y)):
        #id入りの学習データ
        X_train_id, X_val_id = X_id.iloc[train_idx], X_id.iloc[val_idx]
        
        #id抜きの学習データ
        X = X_id.drop(id, axis =1)
        X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
        
        #検証データ
        y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

# Fit
        # train_basketsを計算
        train_baskets = X_train_id.groupby(["race_id"])["horse_id"].count().values

        # モデルの学習
        model = ranker.fit(X_train,
                           y_train,
                           group=train_baskets)

# Predict
        # モデルの予測
        y_pred = model.predict(X_val)

        #結果を入れるデータフレームを用意
        pred_df_fold = pd.DataFrame({
           "race_id": X_val_id['race_id'],
           "horse_id": X_val_id['horse_id'],
           "rank":y_val['rank'],
           "pred": y_pred
           })

        #そのデータフレームをリストに保存
        pred_df_list.append(pred_df_fold)

# Rank
    # 各foldの予測結果を結合してpred_dfを作成
    pred_df = pd.concat(pred_df_list, axis=0).reset_index(drop=True)

    #予測した結果をrace_idごとでグループ化し順位をつける
    pred_df['pred_rank'] = pred_df.groupby('race_id')['pred'].rank(method='min', ascending=False)

    #3位以内ならprizeを1にする
    pred_df['rank_prize'] = pred_df['rank'].apply(lambda x: 1 if x <= 3 else 0)
    pred_df['pred_rank_prize'] = pred_df['pred_rank'].apply(lambda x: 1 if x <= 3 else 0)

#Score
    #正確度を計算
    auc_score = metrics.accuracy_score(pred_df['rank_prize'], pred_df['pred_rank_prize'])
        
    #auc_scoresのリストに保存
    auc_scores.append(auc_score)
    
    #f1を計算            
    f1 = f1_score(pred_df['rank_prize'], pred_df['pred_rank_prize'])
    
    #reportを計算
    report = metrics.classification_report(pred_df['rank_prize'], pred_df['pred_rank_prize'])
    report_num = metrics.precision_recall_fscore_support(pred_df['rank_prize'], pred_df['pred_rank_prize'])
    reports.append(report)

    #reportから情報をとる
    precision_scores.append(report_num[0])
    recall_scores.append(report_num[1])
    f1_scores.append(report_num[2])
    support_scores.append(report_num[3])
    
    return f1

In [8]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

[I 2023-03-13 17:35:51,607] A new study created in memory with name: no-name-b1f09ca0-6660-418c-8d4e-454bfac98513


[LightGBM] [Warning] feature_fraction is set=0.2440734162166043, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.2440734162166043
[LightGBM] [Warning] lambda_l1 is set=0.8819967303556155, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.8819967303556155
[LightGBM] [Warning] bagging_fraction is set=0.8877649883654962, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8877649883654962
[LightGBM] [Warning] lambda_l2 is set=7.450048921073175, reg_lambda=0.0 will be ignored. Current value: lambda_l2=7.450048921073175
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] feature_fraction is set=0.2440734162166043, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.2440734162166043
[LightGBM] [Warning] lambda_l1 is set=0.8819967303556155, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.8819967303556155
[LightGBM] [Warning] bagging_fraction i

[I 2023-03-13 17:35:52,721] Trial 0 finished with value: 0.1511111111111111 and parameters: {'num_leaves': 56, 'learning_rate': 0.04507824069093127, 'feature_fraction': 0.2440734162166043, 'bagging_fraction': 0.8877649883654962, 'bagging_freq': 1, 'lambda_l1': 0.8819967303556155, 'lambda_l2': 7.450048921073175}. Best is trial 0 with value: 0.1511111111111111.


[LightGBM] [Warning] feature_fraction is set=0.6727406896093442, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6727406896093442
[LightGBM] [Warning] lambda_l1 is set=2.9392805339498222, reg_alpha=0.0 will be ignored. Current value: lambda_l1=2.9392805339498222
[LightGBM] [Warning] bagging_fraction is set=0.7533685103335381, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7533685103335381
[LightGBM] [Warning] lambda_l2 is set=2.1232688282411334, reg_lambda=0.0 will be ignored. Current value: lambda_l2=2.1232688282411334
[LightGBM] [Warning] bagging_freq is set=9, subsample_freq=0 will be ignored. Current value: bagging_freq=9
[LightGBM] [Warning] feature_fraction is set=0.6727406896093442, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6727406896093442
[LightGBM] [Warning] lambda_l1 is set=2.9392805339498222, reg_alpha=0.0 will be ignored. Current value: lambda_l1=2.9392805339498222
[LightGBM] [Warning] bagging_fraction

[I 2023-03-13 17:35:54,222] Trial 1 finished with value: 0.09333333333333334 and parameters: {'num_leaves': 93, 'learning_rate': 0.02539764855390118, 'feature_fraction': 0.6727406896093442, 'bagging_fraction': 0.7533685103335381, 'bagging_freq': 9, 'lambda_l1': 2.9392805339498222, 'lambda_l2': 2.1232688282411334}. Best is trial 0 with value: 0.1511111111111111.


...

[I 2023-03-13 17:38:56,965] Trial 97 finished with value: 0.0029741863075196406 and parameters: {'num_leaves': 71, 'learning_rate': 0.04586214910762328, 'feature_fraction': 0.7024455747116606, 'bagging_fraction': 0.7694023200947, 'bagging_freq': 5, 'lambda_l1': 2.6800145265870796, 'lambda_l2': 7.1277823373142875}. Best is trial 0 with value: 0.1511111111111111.


[LightGBM] [Warning] feature_fraction is set=0.7433031655901033, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7433031655901033
[LightGBM] [Warning] lambda_l1 is set=3.06793601834976, reg_alpha=0.0 will be ignored. Current value: lambda_l1=3.06793601834976
[LightGBM] [Warning] bagging_fraction is set=0.8068717220307207, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8068717220307207
[LightGBM] [Warning] lambda_l2 is set=2.4545631970812263, reg_lambda=0.0 will be ignored. Current value: lambda_l2=2.4545631970812263
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] feature_fraction is set=0.7433031655901033, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7433031655901033
[LightGBM] [Warning] lambda_l1 is set=3.06793601834976, reg_alpha=0.0 will be ignored. Current value: lambda_l1=3.06793601834976
[LightGBM] [Warning] bagging_fraction is set=

[I 2023-03-13 17:39:00,829] Trial 98 finished with value: 0.0029444444444444444 and parameters: {'num_leaves': 88, 'learning_rate': 0.017199799145611296, 'feature_fraction': 0.7433031655901033, 'bagging_fraction': 0.8068717220307207, 'bagging_freq': 1, 'lambda_l1': 3.06793601834976, 'lambda_l2': 2.4545631970812263}. Best is trial 0 with value: 0.1511111111111111.


[LightGBM] [Warning] feature_fraction is set=0.6686727525750107, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6686727525750107
[LightGBM] [Warning] lambda_l1 is set=2.3866643452435334, reg_alpha=0.0 will be ignored. Current value: lambda_l1=2.3866643452435334
[LightGBM] [Warning] bagging_fraction is set=0.8335983277007254, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8335983277007254
[LightGBM] [Warning] lambda_l2 is set=6.688933902203732, reg_lambda=0.0 will be ignored. Current value: lambda_l2=6.688933902203732
[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 will be ignored. Current value: bagging_freq=4
[LightGBM] [Warning] feature_fraction is set=0.6686727525750107, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6686727525750107
[LightGBM] [Warning] lambda_l1 is set=2.3866643452435334, reg_alpha=0.0 will be ignored. Current value: lambda_l1=2.3866643452435334
[LightGBM] [Warning] bagging_fraction i

[I 2023-03-13 17:39:04,078] Trial 99 finished with value: 0.002915291529152915 and parameters: {'num_leaves': 52, 'learning_rate': 0.02444542358898852, 'feature_fraction': 0.6686727525750107, 'bagging_fraction': 0.8335983277007254, 'bagging_freq': 4, 'lambda_l1': 2.3866643452435334, 'lambda_l2': 6.688933902203732}. Best is trial 0 with value: 0.1511111111111111.


In [9]:
study.trials_dataframe()

,number,value,datetime_start,datetime_complete,duration,params_bagging_fraction,params_bagging_freq,params_feature_fraction,params_lambda_l1,params_lambda_l2,params_learning_rate,params_num_leaves,state
0,0,0.151111,2023-03-13 17:35:51.610258,2023-03-13 17:35:52.720336,0 days 00:00:01.110078,0.887765,1,0.244073,0.881997,7.450049,0.045078,56,COMPLETE
1,1,0.093333,2023-03-13 17:35:52.722335,2023-03-13 17:35:54.221011,0 days 00:00:01.498676,0.753369,9,0.672741,2.939281,2.123269,0.025398,93,COMPLETE
2,2,0.070000,2023-03-13 17:35:54.223011,2023-03-13 17:35:54.781252,0 days 00:00:00.558241,0.195765,2,0.655754,7.835010,8.034668,0.090485,81,COMPLETE
3,3,0.058444,2023-03-13 17:35:54.783252,2023-03-13 17:35:55.311479,0 days 00:00:00.528227,0.157154,6,0.495938,8.987480,4.382765,0.017546,33,COMPLETE
4,4,0.047963,2023-03-13 17:35:55.312479,2023-03-13 17:35:56.426816,0 days 00:00:01.114337,0.435835,1,0.750424,2.284996,7.217420,0.097627,97,COMPLETE
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,95,0.003036,2023-03-13 17:38:47.245910,2023-03-13 17:38:49.947811,0 days 00:00:02.701901,0.168518,3,0.848571,8.192377,7.533478,0.004446,85,COMPLETE
96,96,0.003005,2023-03-13 17:38:49.948817,2023-03-13 17:38:53.538659,0 days 00:00:03.589842,0.890570,2,0.783168,1.571940,6.256473,0.020372,92,COMPLETE
97,97,0.002974,2023-03-13 17:38:53.539706,2023-03-13 17:38:56.965793,0 days 00:00:03.426087,0.769402,5,0.702446,2.680015,7.127782,0.045862,71,COMPLETE
98,98,0.002944,2023-03-13 17:38:56.966794,2023-03-13 17:39:00.829674,0 days 00:00:03.862880,0.806872,1,0.743303,3.067936,2.454563,0.017200,88,COMPLETE


In [10]:
#各ランダムステートの正確度平均と標準偏差を出力
avg_auc_score = np.mean(auc_scores, axis=0)
std_score = np.std(auc_scores, axis=0)

# 平均と重み付き平均を計算する
precision_avg = np.mean(precision_scores, axis=0)
recall_avg = np.mean(recall_scores, axis=0)
f1_avg = np.mean(f1_scores, axis=0)
support_sum = np.sum(support_scores, axis=0)

# precision_weighted = np.average(precision_avg, weights=support_sum)
# recall_weighted = np.average(recall_avg, weights=support_sum)
# f1_weighted = np.average(f1_avg, weights=support_sum)

In [11]:
print(f"Std: {std_score:.4f}")
print(f"Average AUC score: {avg_auc_score:.4f}")
print(f"Average PRE score 0: {precision_avg[0]:.4f}"+f"   1: {precision_avg[1]:.4f}")
print(f"Average REC score 0: {recall_avg[0]:.4f}"+f"   1: {recall_avg[1]:.4f}")
print(f"Average F1  score 0: {f1_avg[0]:.4f}"+f"   1: {f1_avg[1]:.4f}")

Std: 0.0176
Average AUC score: 0.7788
Average PRE score 0: 0.7858   1: 0.1455
Average REC score 0: 0.9880   1: 0.0075
Average F1  score 0: 0.8752   1: 0.0121


In [12]:
print(reports[0])

              precision    recall  f1-score   support

           0       0.77      0.77      0.77      6637
           1       0.15      0.15      0.15      1800

    accuracy                           0.64      8437
   macro avg       0.46      0.46      0.46      8437
weighted avg       0.64      0.64      0.64      8437



In [13]:
#Optunaによる、ベストパラメータの取得
best_params = study.best_params

In [14]:
#結果を入れるリスト
reports = []
auc_scores = []
precision_scores = []
recall_scores = []
f1_scores = []
support_scores = []
#予測結果を入れるリスト
pred_df_list = []

#ランダムステートを変更し、kfoldを繰り返す
for random_state in range(5):
    
    #kflodの設定
    kfold = KFold(n_splits=5, shuffle=True, random_state=random_state)
    
    #kflodで学習用と検証用に分ける
    for fold, (train_idx, val_idx) in enumerate(kfold.split(X_id, y)):
        X_train_id, X_val_id = X_id.iloc[train_idx], X_id.iloc[val_idx]
        
        X = X_id.drop(id, axis =1)
        X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

        # ここでtrain_basketsを計算する
        train_baskets = X_train_id.groupby(["race_id"])["horse_id"].count().values
        
        best_ranker = LGBMRanker(**best_params)

        # モデルの学習
        model = best_ranker.fit(X_train,
                           y_train,
                           group=train_baskets)

        # モデルの予測
        y_pred = model.predict(X_val)

        #結果を入れるデータフレームを用意
        pred_df_fold = pd.DataFrame({
           "race_id": X_val_id['race_id'],
           "horse_id": X_val_id['horse_id'],
           "rank":y_val['rank'],
           "pred": y_pred
           })
        
        #そのデータフレームをリストに保存
        pred_df_list.append(pred_df_fold)

    # 各foldの予測結果を結合してpred_dfを作成
    pred_df = pd.concat(pred_df_list, axis=0).reset_index(drop=True)
    
    #予測した結果をrace_idごとでグループ化し順位をつける
    pred_df['pred_rank'] = pred_df.groupby('race_id')['pred'].rank(method='min', ascending=False)
    
    #3位以内ならprizeを1にする
    pred_df['rank_prize'] = pred_df['rank'].apply(lambda x: 1 if x <= 3 else 0)
    pred_df['pred_rank_prize'] = pred_df['pred_rank'].apply(lambda x: 1 if x <= 3 else 0)
    
    #正確度を計算
    auc_score = metrics.accuracy_score(pred_df['rank_prize'], pred_df['pred_rank_prize'])
    report = metrics.classification_report(pred_df['rank_prize'], pred_df['pred_rank_prize'])
    report_num = metrics.precision_recall_fscore_support(pred_df['rank_prize'], pred_df['pred_rank_prize'])
    #auc_scoresのリストに保存
    auc_scores.append(auc_score)
    
    reports.append(report)
    
    precision_scores.append(report_num[0])
    recall_scores.append(report_num[1])
    f1_scores.append(report_num[2])
    support_scores.append(report_num[3])

[LightGBM] [Warning] feature_fraction is set=0.2440734162166043, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.2440734162166043
[LightGBM] [Warning] lambda_l1 is set=0.8819967303556155, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.8819967303556155
[LightGBM] [Warning] bagging_fraction is set=0.8877649883654962, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8877649883654962
[LightGBM] [Warning] lambda_l2 is set=7.450048921073175, reg_lambda=0.0 will be ignored. Current value: lambda_l2=7.450048921073175
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] feature_fraction is set=0.2440734162166043, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.2440734162166043
[LightGBM] [Warning] lambda_l1 is set=0.8819967303556155, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.8819967303556155
[LightGBM] [Warning] bagging_fraction i

[LightGBM] [Warning] feature_fraction is set=0.2440734162166043, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.2440734162166043
[LightGBM] [Warning] lambda_l1 is set=0.8819967303556155, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.8819967303556155
[LightGBM] [Warning] bagging_fraction is set=0.8877649883654962, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8877649883654962
[LightGBM] [Warning] lambda_l2 is set=7.450048921073175, reg_lambda=0.0 will be ignored. Current value: lambda_l2=7.450048921073175
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] feature_fraction is set=0.2440734162166043, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.2440734162166043
[LightGBM] [Warning] lambda_l1 is set=0.8819967303556155, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.8819967303556155
[LightGBM] [Warning] bagging_fraction i

In [15]:
#各ランダムステートの正確度平均と標準偏差を出力
avg_auc_score = np.mean(auc_scores, axis=0)
std_score = np.std(auc_scores, axis=0)

# 平均と重み付き平均を計算する
precision_avg = np.mean(precision_scores, axis=0)
recall_avg = np.mean(recall_scores, axis=0)
f1_avg = np.mean(f1_scores, axis=0)
support_sum = np.sum(support_scores, axis=0)

In [16]:
print(f"Std: {std_score:.4f}")
print(f"Average AUC score: {avg_auc_score:.4f}")
print(f"Average PRE score 0: {precision_avg[0]:.4f}"+f"   1: {precision_avg[1]:.4f}")
print(f"Average REC score 0: {recall_avg[0]:.4f}"+f"   1: {recall_avg[1]:.4f}")
print(f"Average F1  score 0: {f1_avg[0]:.4f}"+f"   1: {f1_avg[1]:.4f}")

Std: 0.0404
Average AUC score: 0.7197
Average PRE score 0: 0.7803   1: 0.1511
Average REC score 0: 0.8955   1: 0.0714
Average F1  score 0: 0.8329   1: 0.0896


In [18]:
#特徴量の重要度
for i in model.feature_importances_.argsort()[::-1]:
    print(feature[i], model.feature_importances_[i]/model.feature_importances_.sum())

horse_weight 0.18058899796258565
burden_weight 0.13354324874976847
is_mesu 0.08797925541766993
horse_id 0.08334876829042415
weather_rain 0.064826819781441
ground_type 0.0624189664752732
horse_number 0.05741804037784775
horse_weight_dif 0.050194480459344326
ground_status 0.04556399333209854
is_osu 0.04334135951102056
sex_and_age 0.042970920540840896
rider_id 0.03445082422670865
race_round 0.022041118725689944
race_id 0.021300240785330618
frame_number 0.020559362844971292
weather 0.017040192628264495
is_obstacle 0.007593998888683089
burden_weight_rate 0.007408779403593258
is_down 0.004445267642155954
total_horse_number 0.003704389701796629
is_senba 0.0035191702167067978
owner_id 0.0025930727912576402
is_left_right_straight 0.0025930727912576402
where_racecourse 0.0005556584552694943
tamer_id 0.0
distance 0.0
